In [1]:
import sys
import os
sys.path.append('..')

import pydicom

from impit.dicom.rtstruct_to_nifti.convert import convert_rtstruct


dicom_search_path = "/mnt/d/Work/Data/TOPGEAR"
data_path = './data'

In [2]:
data_objects = {}

for root, dirs, files in os.walk(dicom_search_path, topdown=False):
    for name in files:
        f = os.path.join(root, name)
        ds = pydicom.read_file(f, force=True)
        
        if not 'Modality' in ds:
            continue
            
        
        case = ds.PatientID.split(' ')[1]

        if not case in data_objects:
            data_objects[case] = {}
        
        p = f
        if ds.Modality == 'CT':
            p = os.path.dirname(f)
            
        do = {
            'seriesInstanceUID': ds.SeriesInstanceUID, 
            'path': p}
        data_objects[case][ds.Modality] = do
        
        break # Only read one file per dir
        
        
#     for name in dirs:
#         print(os.path.join(root, name))

In [17]:
import SimpleITK as sitk

for case in data_objects:
    
    case_dir = os.path.join(data_path, case)
    if not os.path.exists(case_dir):
        os.makedirs(case_dir)
    do = data_objects[case]
   
    # Convert the CT and RT Struct
    ct_path = do['CT']['path']
    rtstruct_path = do['RTSTRUCT']['path']
    convert_rtstruct(ct_path, rtstruct_path, output_img=os.path.join(case_dir, 'CT.nii.gz'), output_dir=case_dir)


Converting RTStruct: /mnt/d/Work/Data/TOPGEAR/2019-08__Studies/Case 4_Case 4_RTst_2019-08-29_155027_._._n1__00000/2.16.840.1.114362.1.11775105.22396782581.524840148.216.1015.dcm
Using image series: /mnt/d/Work/Data/TOPGEAR/0002-11__Studies/Case 4_Case 4_CT_0002-11-30_000000_._._n183__00000
Output file prefix: Struct_
Output directory: ./data/4
Image series to be converted to: ./data/4/./data/4/CT.nii.gz
Converting structure 0 with name: CTV_3
Converting structure 1 with name: CTV_4
Converting structure 2 with name: CTV_5
Converting structure 3 with name: CTV_2
Converting structure 4 with name: CTV_1
Converting structure 5 with name: SpCord_6
Converting structure 6 with name: Liver_6
Converting structure 7 with name: Kidney_L_6
Converting structure 8 with name: Kidney_R_6
Converting structure 9 with name: Heart_6
Converting structure 10 with name: BilatLung_6
Converting structure 11 with name: Esophagus_6
Converting structure 12 with name: Duodenum_6
Converting structure 13 with name: P

In [1]:
# If we find an 'edited' structure, take that over as the current structure

for root, dirs, files in os.walk(data_path, topdown=False):
        
    for f in files:
        if 'edited' in f:
            edited_from = os.path.join(root, f)
            print(edited_from)
            edited_to = os.path.join(root, f.replace('_edited', ''))
            
            old_from = edited_to
            old_to = os.path.join(root, edited_to.replace('.nii.gz', '_OLD.nii.gz'))
            
            print(f"Will rename {old_from} to {old_to}")
            os.rename(old_from, old_to)
            print(f"Will rename {edited_from} to {edited_to}")
            os.rename(edited_from, edited_to)

NameError: name 'os' is not defined